# Interval Shrinking for Nonlinear Cosmology Constraints (Finite Bounds)

In [1]:
# Install libraries
%pip install sympy
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install ipywidgets
%pip install ipython

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sympy import symbols, Abs, lambdify
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [5]:
# Declare symbolic variables
a, adot, phi, phidot, theta, thetadot, nu, V0, m3 = symbols('a adot phi phidot theta thetadot nu V0 m3', positive=True)

# Define expressions
addot = a * (V0 * (1 - (phi**2 / nu**2))**2 + m3 * theta) - (2 * adot**2) / a
eta_v = Abs(-4 * (1 - 3 * phi**2) / nu**2 / (1 - phi**2 / nu**2)**2)
epsilon_v = 8 * (phi / (nu**2 - phi**2))**2
friedmann_diff = Abs((adot / a)**2 - (1/3) * V0 * (1 - (phi**2 / nu**2))**2)
phidot_constraint = Abs(phidot + (4/3) * V0 * (1 - phi**2 / nu**2)**2 * (phi * a / (nu**2 * adot)))

In [6]:
# Convert symbolic to numeric functions
variables = (a, adot, phi, phidot, theta, thetadot, nu, V0, m3)
f_addot = lambdify(variables, addot, 'numpy')
f_eta_v = lambdify((phi, nu), eta_v, 'numpy')
f_epsilon_v = lambdify((phi, nu), epsilon_v, 'numpy')
f_friedmann = lambdify((a, adot, phi, nu, V0), friedmann_diff, 'numpy')
f_phidot = lambdify((a, adot, phi, phidot, nu, V0), phidot_constraint, 'numpy')

In [7]:
# Finite conservative bounds
initial_bounds = {
    'a':        (1e-6, 1e2),
    'adot':     (1e-6, 1e2),
    'phi':      (1e-6, 1e1),
    'phidot':   (-1e1, 1e1),
    'theta':    (1e-6, 1e2),
    'thetadot': (1e-6, 1e2),
    'nu':       (1e-2, 1e1),
    'V0':       (1e-6, 1e2),
    'm3':       (1e-6, 1e2),
}


In [28]:
# Sampling and shrinking
def sample_and_check_log(bounds, n_samples=10000):
    samples = {}
    for var, (low, high) in bounds.items():
        if low > 0:
            log_low, log_high = np.log10(low), np.log10(high)
            samples[var] = 10 ** np.random.uniform(log_low, log_high, n_samples)
        else:
            samples[var] = np.random.uniform(low, high, n_samples)

    mask = np.ones(n_samples, dtype=bool)

    try:
        mask &= f_addot(**samples) > 0
        eta_vals = f_eta_v(samples['phi'], samples['nu'])
        mask &= np.isfinite(eta_vals) & (eta_vals < 0.1)

        eps_vals = f_epsilon_v(samples['phi'], samples['nu'])
        mask &= np.isfinite(eps_vals) & (eps_vals < 0.1)

        friedmann_vals = f_friedmann(samples['a'], samples['adot'], samples['phi'], samples['nu'], samples['V0'])
        mask &= np.isfinite(friedmann_vals) & (friedmann_vals < 5)

        # Avoid near-zero denominators
        small = 1e-12
        mask &= (samples['adot'] > small) & (samples['nu'] > small)
        phidot_vals = f_phidot(samples['a'], samples['adot'], samples['phi'], samples['phidot'], samples['nu'], samples['V0'])
        mask &= np.isfinite(phidot_vals) & (phidot_vals < 5)
    except Exception:
        mask &= False

    print("valid ratios:")
    print("  addot      >", np.mean(f_addot(**samples) > 0))
    print("  eta_v      <", np.mean(f_eta_v(samples['phi'], samples['nu']) < 0.1))
    print("  epsilon_v  <", np.mean(f_epsilon_v(samples['phi'], samples['nu']) < 0.1))
    print("  Friedmann  <", np.mean(f_friedmann(samples['a'], samples['adot'], samples['phi'], samples['nu'], samples['V0']) < 5))
    print("  phidot     <", np.mean(f_phidot(samples['a'], samples['adot'], samples['phi'], samples['phidot'], samples['nu'], samples['V0']) < 5))

    return samples, mask

def shrink_intervals_log_adaptive_trace(
    bounds, 
    n_samples=10000, 
    shrink_factor=0.9, 
    expand_factor=1.1, 
    min_valid_ratio=0.95, 
    max_iterations=100
):
    current_bounds = bounds.copy()
    history = []

    for it in range(max_iterations):
        samples, mask = sample_and_check_log(current_bounds, n_samples)
        valid_ratio = np.sum(mask) / n_samples

        step_info = {"iteration": it, "valid_ratio": valid_ratio, "bounds": {}}

        # print(f"Iteration {it+1}: valid_ratio = {valid_ratio:.4f}, bounds = {current_bounds}")

        if valid_ratio >= min_valid_ratio:
            # SHRINK: keep only valid region, shrink a bit
            action = "shrink"
            for var in current_bounds:
                values = samples[var][mask]
                if len(values) == 0:
                    continue
                min_val, max_val = np.min(values), np.max(values)
                range_val = max_val - min_val
                margin = (1 - shrink_factor) * range_val / 2
                new_min = max(min_val + margin, 1e-16)
                new_max = max_val - margin
                current_bounds[var] = (new_min, new_max)
        else:
            # EXPAND: try to explore a larger region
            action = "expand"
            for var in current_bounds:
                low, high = current_bounds[var]
                range_val = high - low
                center = (low + high) / 2
                new_range = range_val * expand_factor
                new_low = max(center - new_range / 2, 1e-12)
                new_high = center + new_range / 2
                current_bounds[var] = (new_low, new_high)
        
        for var, (low, high) in current_bounds.items():
            step_info["bounds"][var] = (low, high)
        step_info["action"] = action
        history.append(step_info)
        

    return current_bounds, history

In [30]:
# Run adaptive shrinking/expansion and track history
final_bounds, history = shrink_intervals_log_adaptive_trace(initial_bounds, max_iterations=100)

# Convert to DataFrame
df_final = pd.DataFrame(final_bounds).T.rename(columns={0: 'min', 1: 'max'})

# Add initial bounds to compare
df_final['init_min'] = [initial_bounds[v][0] for v in df_final.index]
df_final['init_max'] = [initial_bounds[v][1] for v in df_final.index]

# Compute how much each bound changed
df_final['min_ratio'] = df_final['min'] / df_final['init_min']
df_final['max_ratio'] = df_final['max'] / df_final['init_max']

# Compute shrink/expand factor for interval size
df_final['initial_range'] = df_final['init_max'] - df_final['init_min']
df_final['final_range'] = df_final['max'] - df_final['min']
df_final['range_factor'] = df_final['final_range'] / df_final['initial_range']

# Clean display
display_cols = ['min', 'max', 'init_min', 'init_max', 'range_factor']
df_final[display_cols]


valid ratios:
  addot      > 0.4318
  eta_v      < 0.1262
  epsilon_v  < 0.7235
  Friedmann  < 0.4682
  phidot     < 0.6269
valid ratios:
  addot      > 0.5532
  eta_v      < 0.2647
  epsilon_v  < 0.2526
  Friedmann  < 0.3037
  phidot     < 0.4162
valid ratios:
  addot      > 0.5537
  eta_v      < 0.2764
  epsilon_v  < 0.2511
  Friedmann  < 0.2938
  phidot     < 0.4143
valid ratios:
  addot      > 0.5576
  eta_v      < 0.269
  epsilon_v  < 0.2609
  Friedmann  < 0.3032
  phidot     < 0.4161
valid ratios:
  addot      > 0.5578
  eta_v      < 0.2794
  epsilon_v  < 0.2652
  Friedmann  < 0.2979
  phidot     < 0.4155
valid ratios:
  addot      > 0.5541
  eta_v      < 0.276
  epsilon_v  < 0.2644
  Friedmann  < 0.3097
  phidot     < 0.42
valid ratios:
  addot      > 0.5548
  eta_v      < 0.2722
  epsilon_v  < 0.263
  Friedmann  < 0.3034
  phidot     < 0.4143
valid ratios:
  addot      > 0.554
  eta_v      < 0.2809
  epsilon_v  < 0.2701
  Friedmann  < 0.2992
  phidot     < 0.4096
valid ratios:


,min,max,init_min,init_max,range_factor
a,1.000000e-12,13150.125778,0.000001,100.0,131.501259
adot,1.000000e-12,13150.125778,0.000001,100.0,131.501259
phi,1.000000e-12,1315.012572,0.000001,10.0,131.501270
phidot,1.000000e-12,1377.632225,-10.000000,10.0,68.881611
theta,1.000000e-12,13150.125778,0.000001,100.0,131.501259
thetadot,1.000000e-12,13150.125778,0.000001,100.0,131.501259
nu,1.000000e-12,1314.386382,0.100000,10.0,132.766301
V0,1.000000e-12,13150.125778,0.000001,100.0,131.501259
m3,1.000000e-12,13150.125778,0.000001,100.0,131.501259


In [27]:
def interactive_interval_plot(history):
    vars_list = list(history[0]['bounds'].keys())

    def plot_at_step(step):
        step_data = history[step]
        fig, ax = plt.subplots(figsize=(10, 0.5 * len(vars_list)))
        actions = {"shrink": "green", "expand": "red"}

        for i, var in enumerate(vars_list):
            lo, hi = step_data['bounds'][var]
            ax.hlines(i, lo, hi, linewidth=8, color=actions[step_data["action"]])
            ax.text(hi * 1.01, i, f"[{lo:.2e}, {hi:.2e}]", va='center')

        ax.set_yticks(range(len(vars_list)))
        ax.set_yticklabels(vars_list)
        ax.set_xlabel("Value")
        ax.set_title(f"Step {step+1} — Action: {step_data['action']} (valid ratio = {step_data['valid_ratio']:.2%})")
        ax.grid(True)
        plt.tight_layout()
        plt.show()

    slider = widgets.IntSlider(value=0, min=0, max=len(history) - 1, step=1, description='Step:')
    widgets.interact(plot_at_step, step=slider)

# Example usage:
final_bounds, history = shrink_intervals_log_adaptive_trace(initial_bounds)
interactive_interval_plot(history)


interactive(children=(IntSlider(value=0, description='Step:', max=99), Output()), _dom_classes=('widget-intera…